In [1]:
import pandas as pd
import numpy as np


In [2]:
df_matches = pd.read_pickle('files/match_historical_data.pkl')

In [29]:
#df_matches.sort_values('time', ascending=False, inplace=True)

matches = df_matches[(df_matches['team_name']=='Atletico-Madrid')]
matches = matches[matches['time']<'2020-01-01'].head(10)
                        #   &(df_matches['time']<df_matches)]
                           #.head(10)

In [30]:
matches['new_result'] = np.where(matches['result']=='W', 'win'
         , np.where(matches['result']=='L', 'loss', np.where(matches['result']=='D', 'draw', 'None')))

In [31]:
win_loss = matches.groupby(['new_result','venue']).count().T.iloc[[0]]


In [33]:
win_loss.columns = [str(i[0])+'_'+str(i[1]) for i in win_loss.columns]
win_loss.columns = win_loss.columns.str.lower()
win_loss.reset_index(drop=True, inplace=True)

### Previous matches win, loss total
win_loss_all = matches.groupby(['new_result']).count().T.iloc[[0]]
win_loss_all.reset_index(drop=True, inplace=True)
win_loss = pd.concat([win_loss_all,win_loss], axis=1)

In [17]:
win_loss = matches.groupby(['new_result','venue']).count().T.iloc[[0]]
win_loss.columns = [str(i[0])+'_'+str(i[1]) for i in win_loss.columns]
win_loss.columns = win_loss.columns.str.lower()
win_loss.reset_index(drop=True, inplace=True)

In [18]:
win_loss

,draw_away,draw_home,loss_away,loss_home,loss_neutral,win_away,win_home,win_neutral
0,117,85,140,68,1,161,264,6


In [49]:
def match_win_loss(name:str, date_match:str, n_previous_matches:int, la_liga_df):
     
     """Creates the win-loss-draw total and percentage of n previous matches

     Args:
         name (str): [Name of the team to calculate]
         date_match (str): [Date of the calculation (match to be play)]
         n_previous_matches (int): [Number of previous matches to take into account]
         la_liga_df (DataFrame): [DataFrame of raw matches information from fbref]
     """

     results = pd.DataFrame(columns=['number_of_draw', 'number_of_loss', 'number_of_win'
                    , 'number_of_draw_away', 'number_of_draw_home', 'number_of_loss_away'
                    , 'number_of_loss_home', 'number_of_loss_neutral', 'number_of_win_away'
                    , 'number_of_win_home', 'number_of_win_neutral'], index=[0])
    
     la_liga_df.sort_values('time', ascending=False, inplace=True)

     matches = la_liga_df[(la_liga_df['team_name']==name)
                           &(la_liga_df['time']<date_match)].head(n_previous_matches)
    
     matches['new_result'] = np.where(matches['result']=='W', 'number_of_win'
         , np.where(matches['result']=='L', 'number_of_loss', np.where(matches['result']=='D', 'number_of_draw', 'None')))

    ### Previous matches win, loss and draw by venue (home, away)
     win_loss = matches.groupby(['new_result','venue']).count().T.iloc[[0]]
     win_loss.columns = [str(i[0])+'_'+str(i[1]) for i in win_loss.columns]
     win_loss.columns = win_loss.columns.str.lower()
     win_loss.reset_index(drop=True, inplace=True)

     ### Previous matches win, loss total
     win_loss_all = matches.groupby(['new_result']).count().T.iloc[[0]]
     win_loss_all.reset_index(drop=True, inplace=True)
     win_loss = pd.concat([win_loss_all,win_loss], axis=1)
     results.fillna(win_loss, inplace=True)
     results.fillna(0, inplace=True)

     ### percentages
     results['percentage_win'] = results['number_of_win']/n_previous_matches
     results['percentage_loss'] = results['number_of_loss']/n_previous_matches
     results['percentage_draw'] = results['number_of_draw']/n_previous_matches

     number_of_home_games = results['number_of_win_home']+results['number_of_loss_home']+results['number_of_draw_home']
     results['percentage_home_win'] = results['number_of_win_home']/number_of_home_games
     results['percentage_home_loss'] = results['number_of_loss_home']/number_of_home_games
     results['percentage_home_draw'] = results['number_of_draw_home']/number_of_home_games

     number_of_away_games = results['number_of_win_away']+results['number_of_loss_away']+results['number_of_draw_away']
     results['percentage_away_win'] = results['number_of_win_away']/number_of_away_games
     results['percentage_away_loss'] = results['number_of_loss_away']/number_of_away_games
     results['percentage_away_draw'] = results['number_of_draw_away']/number_of_away_games
     
     results['team_name'] = name
     results['date'] = date_match
     results.set_index(['team_name', 'date'], inplace=True)

     return results

In [50]:
res = match_win_loss('Atletico-Madrid', '2020-12-01', 20, df_matches)


In [ ]:
def team_posicions_simulation():
    